# 🅿️ Dynamic Pricing for Urban Parking Lots — Real-Time Notebook

In [ ]:
!pip install pathway bokeh panel --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.9/68.9 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.6/77.6 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.6/777.6 kB 47.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 63.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.6/244.6 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.4/318.4 kB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 985.8/985.8 kB 54.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import pandas as pd
import numpy as np
import datetime
import pathway as pw
import panel as pn
import bokeh.plotting
from google.colab import files

# Upload dataset
uploaded = files.upload()
df = pd.read_csv(list(uploaded.keys())[0])

Saving modified_dataset.csv to modified_dataset.csv


In [ ]:
# Create timestamp column
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'], format='%d-%m-%Y %H:%M:%S')
df = df.sort_values('Timestamp').reset_index(drop=True)

# Demand-related feature engineering
vehicle_weights = {'car': 1.0, 'bike': 0.5, 'truck': 1.5, 'cycle': 0.3}
traffic_weights = {'low': 0.2, 'average': 0.5, 'high': 1.0}

df['VehicleTypeWeight'] = df['VehicleType'].map(vehicle_weights)
df['TrafficScore'] = df['TrafficConditionNearby'].map(traffic_weights)
df['OccupancyRate'] = df['Occupancy'] / df['Capacity']
df['QueueNorm'] = df['QueueLength'] / df['QueueLength'].max()

# Demand calculation
df['Demand'] = (
    0.4 * df['OccupancyRate'] +
    0.2 * df['QueueNorm'] +
    0.2 * df['IsSpecialDay'] +
    0.1 * df['VehicleTypeWeight'] -
    0.1 * df['TrafficScore']
)
df['NormalizedDemand'] = (df['Demand'] - df['Demand'].min()) / (df['Demand'].max() - df['Demand'].min())
df['Price'] = 10 * (1 + 1.2 * df['NormalizedDemand'])
df['Price'] = df['Price'].clip(lower=5, upper=20)

# Save subset to CSV
df[['SystemCodeNumber','Timestamp','Occupancy','Capacity','QueueLength','TrafficConditionNearby','IsSpecialDay','VehicleType','Price']].to_csv('parking_stream.csv', index=False)

In [ ]:
class ParkingSchema(pw.Schema):
    SystemCodeNumber: str
    Timestamp: str
    Occupancy: int
    Capacity: int
    QueueLength: int
    TrafficConditionNearby: str
    IsSpecialDay: int
    VehicleType: str
    Price: float

In [ ]:
data = pw.demo.replay_csv('parking_stream.csv', schema=ParkingSchema, input_rate=100)
fmt = "%Y-%m-%d %H:%M:%S"
data_stream = data.with_columns(
    t = data.Timestamp.dt.strptime(fmt),
    day = data.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00")
)

windowed = data_stream.windowby(
    pw.this.t,
    instance=pw.this.SystemCodeNumber + "_" + pw.this.day,
    window=pw.temporal.tumbling(datetime.timedelta(days=1)),
    behavior=pw.temporal.exactly_once_behavior()
).reduce(
    t=pw.this._pw_window_end,
    sum_price=pw.reducers.sum(pw.this.Price),
    count_price=pw.reducers.count(),
).with_columns(
    avg_price = pw.this.sum_price / pw.this.count_price
)

In [ ]:
pn.extension()
def plot_price(source):
    fig = bokeh.plotting.figure(height=400, width=800, x_axis_type='datetime', title='📈 Avg Daily Parking Price')
    fig.line('t', 'avg_price', source=source, line_width=2, color='green')
    fig.circle('t', 'avg_price', source=source, size=6, color='red')
    return fig

viz = windowed.plot(plot_price, sorting_col='t')
pn.Column(viz).servable()

Column
    [0] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)

In [ ]:
%%capture --no-display
pw.run()

Output()